In [4]:
import sys
import os

sys.path.append(os.path.abspath(".."))


from tiny_graph.graph.graph import Graph
from tiny_graph.constants import START, END

# Create a simple example graph
graph = Graph()


# Define some example actions
def start_action():
    print("Starting workflow")


def process_data():
    print("Processing data")


def validate():
    print("Validating results")


def end_action():
    print("Workflow complete")


# Add nodes to graph
graph.add_node("process", process_data)
graph.add_node("validate", validate)

# Add edges to create workflow
graph.add_edge(START, "process")
graph.add_edge("process", "validate")
graph.add_edge("validate", END)

# Visualize the graph
graph.visualize("example_workflow")